In [89]:
#import dependencies
import pandas as pd
import numpy as np

In [90]:
file = "raw_data/students_complete.csv"
df1 = pd.read_csv(file)
df1.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [91]:
file = "raw_data/schools_complete.csv"
df2 = pd.read_csv(file)
df2 = df2.rename(index=str,columns={"name":"school"})
df2.head()

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [92]:
merge_table = pd.merge(df1,df2,on="school",how="left")
merge_table

school = np.unique(merge_table['school'])
school

#Count of Total Students
totalstudents = merge_table.groupby("type",as_index=False)["Student ID"].count()
totalstudentsdf = pd.DataFrame(totalstudents)
totalstudentsdf = totalstudentsdf.rename(index=str,columns={'Student ID':'Number of Students'})
totalstudentsdf

#Count Distinct Total School IDs
totalschools = merge_table.groupby("type",as_index = False)["School ID"].nunique()
totalschoolsdf = pd.DataFrame(totalschools)
totalschoolsdf = totalschoolsdf.rename(index=str,columns={'School ID':'Number of Schools'})
totalschoolsdf.insert(loc=0,column='type',value=['Charter','District'])
totalschoolsdf

totalbudget = df2.groupby("type",as_index = False)["budget"].sum()
totalbudgetdf = pd.DataFrame(totalbudget)
totalbudgetdf = totalbudgetdf.rename(index = str,columns={'budget':'total_budget'})
totalbudgetdf

averagemath = merge_table.groupby("type",as_index = False)["math_score"].mean()
averagemathdf = pd.DataFrame(averagemath)
averagemathdf = averagemathdf.rename(index = str,columns={'math_score':'average_math_score'})
averagemathdf

averagereading = merge_table.groupby("type",as_index = False)["reading_score"].mean()
averagereadingdf = pd.DataFrame(averagereading)
averagereadingdf = averagereadingdf.rename(index=str,columns={'reading_score':'average_reading_score'})
averagereadingdf

#Set 70% as Passrate
passed_math = merge_table.query('math_score>=70')
passed_reading = merge_table.query('reading_score>=70')
passed_math_count = passed_math.groupby("type",as_index = False)["math_score"].count()
passed_math_countdf = pd.DataFrame(passed_math_count)
passed_math_countdf 

passed_reading_count = passed_reading.groupby("type",as_index = False)["reading_score"].count()
passed_reading_countdf = pd.DataFrame(passed_reading_count)
passed_reading_countdf

#total_math_count
total_math_count = merge_table.groupby("type",as_index = False)["math_score"].count()
total_math_countdf = pd.DataFrame(total_math_count)
total_math_countdf = total_math_countdf.rename(index=str,columns={'math_score':'Total_Math_Count'})
total_math_countdf

#total_reading_count
total_reading_count = merge_table.groupby("type",as_index = False)["reading_score"].count()
total_reading_countdf = pd.DataFrame(total_reading_count)
total_reading_countdf = total_reading_countdf.rename(index=str,columns={'reading_score':'Total_Reading_Count'})
total_reading_countdf

#Pass Rate
percent_passing_math = (passed_math_count['math_score'] /  total_math_count['math_score'])*100
percent_passing_mathdf = pd.DataFrame(percent_passing_math)
percent_passing_mathdf.insert(loc=0,column='type',value=['Charter','District'])
percent_passing_mathdf = percent_passing_mathdf.rename(index=str,columns ={'math_score':'Percent_Passing_Math'})
percent_passing_mathdf
percent_passing_reading = (passed_reading_count['reading_score'] / total_reading_count['reading_score']) * 100
percent_passing_readingdf = pd.DataFrame(percent_passing_reading,columns = {'type','reading_score'})
percent_passing_readingdf = percent_passing_readingdf.rename(index=str,columns = {'reading_score':'Percent_Passing_Reading'})
percent_passing_readingdf = percent_passing_readingdf.set_value('1','type','District')
percent_passing_readingdf = percent_passing_readingdf.set_value('0','type','Charter')
percent_passing_readingdf
Overall_Passing_Rate = (percent_passing_mathdf['Percent_Passing_Math'] + percent_passing_readingdf['Percent_Passing_Reading']) / 2
Overall_Passing_Ratedf = pd.DataFrame(Overall_Passing_Rate,columns = {'Overall_Passing_Rate'})
Overall_Passing_Ratedf.insert(loc=0,column='type',value=['Charter','District'])
Overall_Passing_Ratedf


,type,Overall_Passing_Rate
0,Charter,95.173856
1,District,73.711818


#total count math
total_math_count =merge_table.groupby("type",as_index = False)["math_score"].count()
total_math_countdf = pd.DataFrame(total_math_count)
total_math_countdf

In [93]:
#District Summary Snapshot

dmerge1 = pd.merge(totalschoolsdf,totalstudentsdf,on = "type",how="left")
dmerge2 = pd.merge(dmerge1,totalbudgetdf,on="type",how="left")
dmerge3 = pd.merge(dmerge2,averagemathdf,on="type",how="left")
dmerge4 = pd.merge(dmerge3,averagereadingdf,on="type",how="left")
dmerge5 = pd.merge(dmerge4,percent_passing_mathdf,on="type",how="left")
dmerge6 = pd.merge(dmerge5,percent_passing_readingdf,on="type",how="left")
dmerge7 = pd.merge(dmerge6,Overall_Passing_Ratedf,on="type",how="left")
dmerge7 = dmerge7.drop(dmerge7.index[0])
dmerge7

,type,Number of Schools,Number of Students,total_budget,average_math_score,average_reading_score,Percent_Passing_Math,Percent_Passing_Reading,Overall_Passing_Rate
1,District,7,26976,17347923,76.987026,80.962485,66.518387,80.905249,73.711818


In [94]:
#School Metrics
Student_Count = merge_table.groupby(["School ID","school"],as_index = False)["Student ID"].count()
Student_Count = Student_Count.rename(index=str,columns = {'Student ID':'Total Students'})

school_total_students = Student_Count['Total Students']
school_total_students = pd.to_numeric(school_total_students,errors='coerce',downcast='float').tolist()
school_total_students


[2917.0,
 2949.0,
 1761.0,
 4635.0,
 1468.0,
 2283.0,
 1858.0,
 4976.0,
 427.0,
 962.0,
 1800.0,
 3999.0,
 4761.0,
 2739.0,
 1635.0]

In [95]:
Student_Budget = df2.groupby(["School ID","school","type"],as_index = False)["budget"].sum()
Student_Budget = Student_Budget.rename(index=str,columns = {'budget':'Total School Budget'})
Student_Budget['Total_Students'] = school_total_students
Student_Budget

,School ID,school,type,Total School Budget,Total_Students
0,0,Huang High School,District,1910635,2917.0
1,1,Figueroa High School,District,1884411,2949.0
2,2,Shelton High School,Charter,1056600,1761.0
3,3,Hernandez High School,District,3022020,4635.0
4,4,Griffin High School,Charter,917500,1468.0
5,5,Wilson High School,Charter,1319574,2283.0
6,6,Cabrera High School,Charter,1081356,1858.0
7,7,Bailey High School,District,3124928,4976.0
8,8,Holden High School,Charter,248087,427.0
9,9,Pena High School,Charter,585858,962.0


In [96]:
Per_Student_Budget = Student_Budget['Total School Budget'] / Student_Count['Total Students']
Per_Student_Budget = pd.DataFrame(Per_Student_Budget)
Per_Student_Budget = Student_Budget.join(Per_Student_Budget,how="outer")
Per_Student_Budget = Per_Student_Budget.rename(index = str,columns = {0:"Per Student Budget"})

In [97]:
AverageReadingScore = merge_table.groupby(["School ID","school"],as_index = False)["reading_score"].mean()
AverageReadingScore = AverageReadingScore[['School ID','reading_score']]
AverageReadingScore

,School ID,reading_score
0,0,81.182722
1,1,81.158020
2,2,83.725724
3,3,80.934412
4,4,83.816757
5,5,83.989488
6,6,83.975780
7,7,81.033963
8,8,83.814988
9,9,84.044699


In [98]:
AverageMathScore = merge_table.groupby(["School ID","school","type"],as_index = False)["math_score"].mean()
AverageMathScore = AverageMathScore[['School ID','math_score']]
Per_Student_Budget = pd.merge(Per_Student_Budget,AverageReadingScore,on="School ID",how = "left")
Per_Student_Budget = Per_Student_Budget.rename(index=str,columns={'reading_score':'Average_Reading_Score'})
Per_Student_Budget

,School ID,school,type,Total School Budget,Total_Students,Per Student Budget,Average_Reading_Score
0,0,Huang High School,District,1910635,2917.0,655.0,81.182722
1,1,Figueroa High School,District,1884411,2949.0,639.0,81.158020
2,2,Shelton High School,Charter,1056600,1761.0,600.0,83.725724
3,3,Hernandez High School,District,3022020,4635.0,652.0,80.934412
4,4,Griffin High School,Charter,917500,1468.0,625.0,83.816757
5,5,Wilson High School,Charter,1319574,2283.0,578.0,83.989488
6,6,Cabrera High School,Charter,1081356,1858.0,582.0,83.975780
7,7,Bailey High School,District,3124928,4976.0,628.0,81.033963
8,8,Holden High School,Charter,248087,427.0,581.0,83.814988
9,9,Pena High School,Charter,585858,962.0,609.0,84.044699


In [99]:
Score = pd.merge(Per_Student_Budget,AverageMathScore,on="School ID",how = "left")
Score = Score.rename(index=str,columns={'math_score':'Average_Math_Score'})
Score

,School ID,school,type,Total School Budget,Total_Students,Per Student Budget,Average_Reading_Score,Average_Math_Score
0,0,Huang High School,District,1910635,2917.0,655.0,81.182722,76.629414
1,1,Figueroa High School,District,1884411,2949.0,639.0,81.158020,76.711767
2,2,Shelton High School,Charter,1056600,1761.0,600.0,83.725724,83.359455
3,3,Hernandez High School,District,3022020,4635.0,652.0,80.934412,77.289752
4,4,Griffin High School,Charter,917500,1468.0,625.0,83.816757,83.351499
5,5,Wilson High School,Charter,1319574,2283.0,578.0,83.989488,83.274201
6,6,Cabrera High School,Charter,1081356,1858.0,582.0,83.975780,83.061895
7,7,Bailey High School,District,3124928,4976.0,628.0,81.033963,77.048432
8,8,Holden High School,Charter,248087,427.0,581.0,83.814988,83.803279
9,9,Pena High School,Charter,585858,962.0,609.0,84.044699,83.839917


In [100]:
school_passed_math = passed_math.groupby("School ID",as_index=False)["math_score"].count()
school_passed_reading = passed_reading.groupby("School ID",as_index=False)["reading_score"].count()
school_passed_reading = school_passed_reading['reading_score']
school_passed_math = school_passed_math['math_score']
school_passed_math = pd.to_numeric(school_passed_math,errors='coerce',downcast='float').tolist()
school_passed_math
percent_passed_math = np.array(school_passed_math) / np.array(school_total_students)
percent_passed_math
percent_passed_reading = np.array(school_passed_reading) / np.array(school_total_students)
percent_passed_reading




array([ 0.81316421,  0.80739234,  0.95854628,  0.80862999,  0.97138965,
        0.96539641,  0.97039828,  0.8193328 ,  0.96252927,  0.95945946,
        0.96611111,  0.80220055,  0.81222432,  0.79299014,  0.97308869])

In [101]:
#School Summary
Score['Percent_Passed_Math'] = percent_passed_math*100
Score['Percent_Passed_Reading'] = percent_passed_reading*100
Score['Overall_Passing_Rate'] = (percent_passed_math + percent_passed_reading) / 2 * 100
Score = Score.sort_values('Overall_Passing_Rate')
Score


,School ID,school,type,Total School Budget,Total_Students,Per Student Budget,Average_Reading_Score,Average_Math_Score,Percent_Passed_Math,Percent_Passed_Reading,Overall_Passing_Rate
11,11,Rodriguez High School,District,2547363,3999.0,637.0,80.744686,76.842711,66.366592,80.220055,73.293323
1,1,Figueroa High School,District,1884411,2949.0,639.0,81.158020,76.711767,65.988471,80.739234,73.363852
0,0,Huang High School,District,1910635,2917.0,655.0,81.182722,76.629414,65.683922,81.316421,73.500171
12,12,Johnson High School,District,3094650,4761.0,650.0,80.966394,77.072464,66.057551,81.222432,73.639992
13,13,Ford High School,District,1763916,2739.0,644.0,80.746258,77.102592,68.309602,79.299014,73.804308
3,3,Hernandez High School,District,3022020,4635.0,652.0,80.934412,77.289752,66.752967,80.862999,73.807983
7,7,Bailey High School,District,3124928,4976.0,628.0,81.033963,77.048432,66.680064,81.933280,74.306672
8,8,Holden High School,Charter,248087,427.0,581.0,83.814988,83.803279,92.505855,96.252927,94.379391
2,2,Shelton High School,Charter,1056600,1761.0,600.0,83.725724,83.359455,93.867121,95.854628,94.860875
10,10,Wright High School,Charter,1049400,1800.0,583.0,83.955000,83.682222,93.333333,96.611111,94.972222


In [102]:
#Top 5 Performing Schools
Top_5 = Score.tail(5)
Top_5

,School ID,school,type,Total School Budget,Total_Students,Per Student Budget,Average_Reading_Score,Average_Math_Score,Percent_Passed_Math,Percent_Passed_Reading,Overall_Passing_Rate
5,5,Wilson High School,Charter,1319574,2283.0,578.0,83.989488,83.274201,93.867718,96.539641,95.203679
4,4,Griffin High School,Charter,917500,1468.0,625.0,83.816757,83.351499,93.392371,97.138965,95.265668
9,9,Pena High School,Charter,585858,962.0,609.0,84.044699,83.839917,94.594595,95.945946,95.270270
14,14,Thomas High School,Charter,1043130,1635.0,638.0,83.848930,83.418349,93.272171,97.308869,95.290520
6,6,Cabrera High School,Charter,1081356,1858.0,582.0,83.975780,83.061895,94.133477,97.039828,95.586652


In [103]:
#Bottom 5 Performing Schools
Bottom_5 = Score.head(5)
Bottom_5

,School ID,school,type,Total School Budget,Total_Students,Per Student Budget,Average_Reading_Score,Average_Math_Score,Percent_Passed_Math,Percent_Passed_Reading,Overall_Passing_Rate
11,11,Rodriguez High School,District,2547363,3999.0,637.0,80.744686,76.842711,66.366592,80.220055,73.293323
1,1,Figueroa High School,District,1884411,2949.0,639.0,81.158020,76.711767,65.988471,80.739234,73.363852
0,0,Huang High School,District,1910635,2917.0,655.0,81.182722,76.629414,65.683922,81.316421,73.500171
12,12,Johnson High School,District,3094650,4761.0,650.0,80.966394,77.072464,66.057551,81.222432,73.639992
13,13,Ford High School,District,1763916,2739.0,644.0,80.746258,77.102592,68.309602,79.299014,73.804308


In [104]:
#Math Scores By Grade By School
MathScoreByGrade = merge_table.groupby(["grade","school"],as_index=False)["math_score"].mean()
MathScoreByGrade = MathScoreByGrade.rename(index = str,columns={"math_score":"Average_Math_Score"})
MathScoreByGrade

,grade,school,Average_Math_Score
0,10th,Bailey High School,76.996772
1,10th,Cabrera High School,83.154506
2,10th,Figueroa High School,76.539974
3,10th,Ford High School,77.672316
4,10th,Griffin High School,84.229064
5,10th,Hernandez High School,77.337408
6,10th,Holden High School,83.429825
7,10th,Huang High School,75.908735
8,10th,Johnson High School,76.691117
9,10th,Pena High School,83.372000


In [105]:
#Reading Scores By Grade By School
ReadingScoreByGrade = merge_table.groupby(["grade","school"],as_index=False)["reading_score"].mean()
ReadingScoreByGrade = ReadingScoreByGrade.rename(index = str,columns={"reading_score":"Average_Reading_Score"})
ReadingScoreByGrade

,grade,school,Average_Reading_Score
0,10th,Bailey High School,80.907183
1,10th,Cabrera High School,84.253219
2,10th,Figueroa High School,81.408912
3,10th,Ford High School,81.262712
4,10th,Griffin High School,83.706897
5,10th,Hernandez High School,80.660147
6,10th,Holden High School,83.324561
7,10th,Huang High School,81.512386
8,10th,Johnson High School,80.773431
9,10th,Pena High School,83.612000


In [106]:
#Scores By School Spending
bins = [550,580,610,640,670]
group_names = ['550-580','581-610','611-640','641-670']
Score['Spending_Rank'] = pd.cut(Score['Per Student Budget'],bins,labels=group_names)
Spending = Score.groupby(["Spending_Rank"],as_index = False)["Average_Math_Score"].mean()
Spending2= Score.groupby(["Spending_Rank"],as_index = False)["Average_Reading_Score"].mean()
Spending3 = Score.groupby(["Spending_Rank"],as_index = False)["Percent_Passed_Math"].mean()
Spending4 = Score.groupby(["Spending_Rank"],as_index = False)["Percent_Passed_Reading"].mean()
Spending = pd.merge(Spending,Spending2,on="Spending_Rank",how="left")
Spending = pd.merge(Spending,Spending3,on="Spending_Rank",how="left")
Spending = pd.merge(Spending,Spending4,on="Spending_Rank",how="left")
Spending['Overall_Passing_Rate'] = (Spending['Percent_Passed_Math'] + Spending['Percent_Passed_Reading']) / 2
Spending

,Spending_Rank,Average_Math_Score,Average_Reading_Score,Percent_Passed_Math,Percent_Passed_Reading,Overall_Passing_Rate
0,550-580,83.274201,83.989488,93.867718,96.539641,95.203679
1,581-610,83.549353,83.903238,93.686876,96.340888,95.013882
2,611-640,79.474551,82.120471,77.139934,87.468080,82.304007
3,641-670,77.023555,80.957446,66.701010,80.675217,73.688113


In [107]:
#Scores By School Size
bins = [0,2000,4000,6000]
group_names = ['small','medium','large']
Score['Size_Rank'] = pd.cut(Score['Total_Students'],bins,labels=group_names)
Score = Score[['School ID','school','type','Total School Budget','Total_Students','Per Student Budget','Average_Math_Score','Average_Reading_Score','Percent_Passed_Math','Percent_Passed_Reading','Size_Rank']]
School = Score.groupby(["Size_Rank"],as_index = False)["Average_Math_Score"].mean()
School2= Score.groupby(["Size_Rank"],as_index = False)["Average_Reading_Score"].mean()
School3 = Score.groupby(["Size_Rank"],as_index = False)["Percent_Passed_Math"].mean()
School4 = Score.groupby(["Size_Rank"],as_index = False)["Percent_Passed_Reading"].mean()
School = pd.merge(School,School2,on="Size_Rank",how="left")
School = pd.merge(School,School3,on="Size_Rank",how="left")
School = pd.merge(School,School4,on="Size_Rank",how="left")
School['Overall_Passing_Rate'] = (Spending['Percent_Passed_Math'] + Spending['Percent_Passed_Reading']) / 2
School

,Size_Rank,Average_Math_Score,Average_Reading_Score,Percent_Passed_Math,Percent_Passed_Reading,Overall_Passing_Rate
0,small,83.502373,83.883125,93.585560,96.593182,95.203679
1,medium,78.112137,81.564235,72.043261,83.622873,95.013882
2,large,77.136883,80.978256,66.496861,81.339570,82.304007


In [108]:
#Scores By School Type
Type = Score.groupby('type',as_index=False)['Average_Math_Score'].mean()
Type2 = Score.groupby('type',as_index=False)['Average_Reading_Score'].mean()
Type3 = Score.groupby('type',as_index=False)['Percent_Passed_Math'].mean()
Type4 = Score.groupby('type',as_index=False)['Percent_Passed_Reading'].mean()
Type = pd.merge(Type,Type2,on="type",how="left")
Type = pd.merge(Type,Type3,on="type",how="left")
Type = pd.merge(Type,Type4,on="type",how="left")
Type['Overall_Passing_Rate'] = (Type['Percent_Passed_Math'] + Type['Percent_Passed_Reading'])/2
Type

,type,Average_Math_Score,Average_Reading_Score,Percent_Passed_Math,Percent_Passed_Reading,Overall_Passing_Rate
0,Charter,83.473852,83.896421,93.620830,96.586489,95.103660
1,District,76.956733,80.966636,66.548453,80.799062,73.673757
